# Storm Research Assistant

Reference
https://github.com/langchain-ai/langgraph/blob/main/examples/storm/storm.ipynb


In [1]:
## Prereqs

In [2]:

# %pip install -U langchain_community langchain_openai langgraph wikipedia  scikit-learn  langchain_fireworks langchain_anthropic
# # We use one or the other search engine below
# %pip install -U tavily-python, playwright
# %pip install -U duckduckgo-search
# # ! apt-get install graphviz graphviz-dev
# %pip install pygraphviz



In [3]:

example_topic = "Impact of THE Red Cross Church IN Zimbabwe early history"

In [4]:
from storm import *

os.environ["LANGCHAIN_PROJECT"] = 'STORM_RESEARCHER'


fast_llm, _ = get_openai_llms(regular_model="gpt-3.5-turbo", long_context_model="gpt-3.5-turbo-0125")

_, long_context_llm = get_anthropic_llms()

# ollama_model = 'mistral:7b-instruct-q4_K_M'
# fast_llm, long_context_llm = get_ollama_llms(regular_model=ollama_model, long_context_model=ollama_model)


embeddings = get_gpt4all_embeddings()

_ = None


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


### Generate Initial Outline


In [5]:
outline_system_prompt = prompts.outline_system_wiki_writer
outline_human_prompt = prompts.outline_user_topic_formatinstructions
direct_gen_outline_prompt = get_chat_prompt_from_prompt_templates([outline_system_prompt, outline_human_prompt])

outline_parser = get_pydantic_parser(pydantic_object=Outline)
generate_outline_direct = get_chain_with_outputparser(direct_gen_outline_prompt, fast_llm, outline_parser)

In [6]:

initial_outline = generate_outline_direct.invoke({"topic": example_topic})
logger.info(initial_outline.as_str)

2024-03-31 14:42:27,820 [MainThread  ] [INFO ]  # Impact of The Red Cross Church in Zimbabwe Early History

## Introduction to The Red Cross Church in Zimbabwe

Overview of the establishment and early history of The Red Cross Church in Zimbabwe.

### Founding of The Red Cross Church in Zimbabwe

Information on the founders and the initial mission of the church in Zimbabwe.

### Early Activities and Programs

Details about the early activities, charity work, and community programs initiated by the church.

## Impact on Zimbabwean Society

Exploration of the significant impact of The Red Cross Church on the society and culture of Zimbabwe.

### Humanitarian Efforts

Discussion on the humanitarian aid provided by the church during critical times in Zimbabwe.

### Educational Initiatives

Information on the church's role in promoting education and literacy in Zimbabwe.

## Challenges and Controversies

Analysis of the challenges faced by The Red Cross Church in Zimbabwe and any controversi

### Expand Topics


In [7]:
related_subjects_prompt = get_chat_prompt_from_prompt_templates([prompts.related_subjects_human_wiki_writer])
related_topics_parser = get_pydantic_parser(RelatedSubjects)
expand_chain = get_chain_with_outputparser(related_subjects_prompt, fast_llm, related_topics_parser)

In [8]:
related_subjects = await expand_chain.ainvoke({"topic": example_topic})
related_subjects

RelatedSubjects(topics=['Red Cross in Zimbabwe', 'History of Christianity in Zimbabwe', 'Impact of missionary work in Zimbabwe', 'Religion in Zimbabwe', 'Colonialism in Zimbabwe', 'Humanitarian aid in Zimbabwe'])

## Generate Perspectives

From these related subjects, we can select representative Wiki editors as "subject matter experts" with distinct backgrounds and affiliations. These will help distribute the search process to encourage a more well-rounded final report.


In [9]:


gen_perspectives_prompt = get_chat_prompt_from_prompt_templates([prompts.perspective_system_generator, prompts.outline_user_topic_formatinstructions])
perspectives_parser = get_pydantic_parser(pydantic_object=Perspectives)

gen_perspectives_chain = gen_perspectives_prompt.partial(format_instructions=perspectives_parser.get_format_instructions()) | fast_llm | perspectives_parser



@as_runnable
async def survey_subjects(topic: str)-> Perspectives:
    logger.info(f"Survey Subjects for Topic: {topic}")
    related_subjects = await expand_chain.ainvoke({"topic": topic})
    retrieved_docs = await wikipedia_retriever.abatch(
        related_subjects.topics, return_exceptions=True
    )
    all_docs = []
    for docs in retrieved_docs:
        if isinstance(docs, BaseException):
            continue
        all_docs.extend(docs)
    logger.info(f"Retrieved {len(all_docs)} docs for Topic: {topic}")
    
    formatted = format_docs(all_docs)
    return await gen_perspectives_chain.ainvoke({"examples": formatted, "topic": topic})


In [10]:
perspectives = await survey_subjects.ainvoke(example_topic)
perspectives.dict()

2024-03-31 14:42:28,899 [MainThread  ] [INFO ]  Survey Subjects for Topic: Impact of THE Red Cross Church IN Zimbabwe early history
2024-03-31 14:42:41,891 [MainThread  ] [INFO ]  Retrieved 6 docs for Topic: Impact of THE Red Cross Church IN Zimbabwe early history


{'editors': [{'affiliation': 'Zimbabwe Historical Society',
   'name': 'Dr. Tendai Moyo',
   'role': 'Historian',
   'description': 'Dr. Moyo is a prominent historian specializing in the early history of Zimbabwe, with a focus on the impact of missionary activities. He will provide insights into the interactions between the Red Cross Church and the historical development of Zimbabwe.'},
  {'affiliation': 'Red Cross Zimbabwe',
   'name': 'Sarah Chikwava',
   'role': 'Humanitarian Worker',
   'description': "Sarah Chikwava has been actively involved in humanitarian work in Zimbabwe, particularly with the Red Cross Church. She will offer firsthand accounts of the Red Cross Church's initiatives and their impact on communities in Zimbabwe."},
  {'affiliation': 'Zimbabwe Christian Council',
   'name': 'Rev. Michael Ngwenya',
   'role': 'Religious Scholar',
   'description': 'Rev. Ngwenya is a respected religious scholar focusing on the history of Christianity in Zimbabwe. He will provide a p

## Expert Dialog

Each wikipedia writer is primed to role-play using the perspectives presented above. It will ask a series of questions of a second "domain expert" with access to a search engine. This generate content to generate a refined outline as well as an updated index of reference documents.

### Interview State

The conversation is cyclic, so we will construct it within its own graph. The State will contain messages, the reference docs, and the editor (with its own "persona") to make it easy to parallelize these conversations.


# Dialog Roles

The graph will have two participants: the wikipedia editor (generate_question), who asks questions based on its assigned role, and a domain expert (`gen_answer_chain), who uses a search engine to answer the questions as accurately as possible.


In [11]:
gen_qn_prompt = get_chat_prompt_from_prompt_templates([prompts.gen_question_system_generator, prompts.generate_messages_placeholder()])


def swap_roles(state: InterviewState, name: str) -> InterviewState:

    # Normalize name
    name = cleanup_name(name)

    logger.info(f'Swapping roles for {name}')

    converted = []
    for message in state["messages"]:
        if isinstance(message, AIMessage) and message.name != name:
            message = HumanMessage(**message.dict(exclude={"type"}))
        converted.append(message)
    
    state['messages'] = converted
    
    logger.info(f'Converted messages for {name} while swapping roles: {len(converted)} messages')
    return state


@as_runnable
async def generate_question(state: InterviewState) -> InterviewState:
    editor: Editor = state["editor"]

    name = cleanup_name(editor.name)


    logger.info(f'Generating question for {name}')

    gn_chain = (
        RunnableLambda(swap_roles).bind(name=name)
        | gen_qn_prompt.partial(persona=editor.persona)
        | fast_llm
        | RunnableLambda(tag_with_name).bind(name=name)
    )
    result:AIMessage = await gn_chain.ainvoke(state)
    state["messages"] = ([result])

    logger.info(f'Generated question for {name}')
    return state

In [12]:
intial_messages = [prompts.initial_question]
initial_state: InterviewState = {
    "editor": perspectives.editors[0],
    "messages": intial_messages,
    "references": None
}

question = await generate_question.ainvoke(initial_state)

question["messages"][0]

2024-03-31 14:42:46,457 [MainThread  ] [INFO ]  Generating question for DrTendaiMoyo
2024-03-31 14:42:46,478 [asyncio_3   ] [INFO ]  Swapping roles for DrTendaiMoyo
2024-03-31 14:42:46,478 [asyncio_3   ] [INFO ]  Converted messages for DrTendaiMoyo while swapping roles: 1 messages
2024-03-31 14:42:47,460 [MainThread  ] [INFO ]  Generated question for DrTendaiMoyo


AIMessage(content='I apologize for any confusion, but I am actually a Wikipedia writer focused on the early history of Zimbabwe, specifically examining the impact of missionary activities. In this context, I am particularly interested in the interactions between the Red Cross Church and the historical development of Zimbabwe. Can you provide insights into how the Red Cross Church influenced events in Zimbabwe during that time?', response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 224, 'total_tokens': 294}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, name='DrTendaiMoyo')

### Answer questions

The `gen_answer_chain` first generates queries (query expansion) to answer the editor's question, then responds with citations.


In [13]:




gen_queries_prompt = get_chat_prompt_from_prompt_templates([prompts.gen_queries_system_generator, prompts.generate_messages_placeholder()])
queries_parser = get_pydantic_parser(Queries)

gen_queries_chain = get_chain_with_outputparser(gen_queries_prompt, fast_llm, queries_parser)
# gen_queries_prompt.partial(format_instructions=queries_parser.get_format_instructions()) | fast_llm | queries_parser

In [14]:

queries = await gen_queries_chain.ainvoke(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)

queries

Queries(queries=['Red Cross Church missionary activities in early history of Zimbabwe', 'Impact of Red Cross Church on historical development of Zimbabwe', 'Interactions between Red Cross Church and Zimbabwe during early history'])

In [15]:
gen_answer_prompt = get_chat_prompt_from_prompt_templates([prompts.generate_answer_system_generator, prompts.generate_messages_placeholder()])
ac_parser = get_pydantic_parser(pydantic_object=AnswerWithCitations)

gen_answer_chain = get_chain_with_outputparser(gen_answer_prompt, fast_llm, ac_parser)\
    .with_config(run_name="GenerateAnswer")\
    # .with_structured_output(AnswerWithCitations, include_raw=True)                                  

In [16]:

async def gen_answer(
    state: InterviewState,
    config: Optional[RunnableConfig] = None,
    name: str = "SubjectMatterExpert",
    max_str_len: int = 15000,
):
    name = cleanup_name(name)

    logger.info(f'START - Generate answers for [{name}]')

    swapped_state = swap_roles(state, name)  # Convert all other AI messages
    
    # Generate search engine queries
    queries:Queries = await gen_queries_chain.ainvoke(swapped_state)

    logger.info(f"Got {len(queries.queries)} search engine queries for [{name}] -\n\t {queries.queries}")

    # Run search engine
    query_results = await search_engine.abatch(
        queries.queries, config, return_exceptions=True
    )
    successful_results = [
        res for res in query_results if not isinstance(res, Exception)
    ]

    all_query_results = {
        res["url"]: res["content"] for results in successful_results for res in results
    }
    
    logger.info(f"Got {len(successful_results)} search engine results for [{name}] - \n\t {all_query_results}")

    # We could be more precise about handling max token length if we wanted to here
    dumped_successful_results = json.dumps(all_query_results)[:max_str_len]
    
    logger.info(f"Dumped {len(dumped_successful_results)} characters for [{name}] - \n\t {dumped_successful_results}")
    
    # Append Questions from Wikipedia and Answers from the search engine
    ai_message_for_queries: AIMessage = get_ai_message(json.dumps(queries.as_dict()))
    
    tool_results_message = generate_human_message(dumped_successful_results)
    
    logger.debug(f"Got {ai_message_for_queries} for [{name}]")
    
    # tool_call = queries["raw"].additional_kwargs["tool_calls"][0]
    # tool_id = tool_call["id"]

    # tool_message = ToolMessage(tool_call_id=tool_id, content=dumped)
    

    swapped_state["messages"].extend([ai_message_for_queries, tool_results_message])
    
    # Only update the shared state with the final answer to avoid
    # polluting the dialogue history with intermediate messages
    try:
        generated: AnswerWithCitations = await gen_answer_chain.ainvoke(swapped_state)
        
        logger.info(f"Genreted final answer {generated} for [{name}] - \n\t {generated.as_str}")

    except Exception as e:
        logger.error(f"Error generating answer for [{name}] - {e}")
        generated = AnswerWithCitations(answer="", cited_urls=[])
    
    cited_urls = set(generated.cited_urls)
    
    # Save the retrieved information to a the shared state for future reference
    cited_references = {k: v for k, v in all_query_results.items() if k in cited_urls}
    
    formatted_message = AIMessage(name=name, content=generated.as_str)
    # Add message to shared state
    # state["messages"].append(formatted_message)
    state["messages"] = add_messages(state["messages"], [formatted_message])
    
    # Update references with cited references
    state["references"] = update_references(state["references"], cited_references)

    logger.info(f'END - generate answer for [{name}]')
    
    return state
    

In [17]:
intial_messages = [prompts.initial_question, generate_human_message(question["messages"][0].content)]

initial_state: InterviewState = {
    "editor": perspectives.editors[0],
    "messages": intial_messages,
    "references": {}
}

example_answer = await gen_answer(initial_state)
example_answer["messages"][-1].content

2024-03-31 14:42:48,718 [MainThread  ] [INFO ]  START - Generate answers for [SubjectMatterExpert]
2024-03-31 14:42:48,719 [MainThread  ] [INFO ]  Swapping roles for SubjectMatterExpert
2024-03-31 14:42:48,720 [MainThread  ] [INFO ]  Converted messages for SubjectMatterExpert while swapping roles: 2 messages
2024-03-31 14:42:49,492 [MainThread  ] [INFO ]  Got 3 search engine queries for [SubjectMatterExpert] -
	 ['Red Cross Church influence on early history of Zimbabwe', 'Missionary activities impact on Zimbabwe', 'Interactions between Red Cross Church and historical development of Zimbabwe']


Searching DuckDuckGo for [Red Cross Church influence on early history of Zimbabwe]
Got search engine results: 5 for [Red Cross Church influence on early history of Zimbabwe]
Searching DuckDuckGo for [Missionary activities impact on Zimbabwe]
Got search engine results: 5 for [Missionary activities impact on Zimbabwe]
Searching DuckDuckGo for [Interactions between Red Cross Church and historical development of Zimbabwe]


2024-03-31 14:42:52,336 [MainThread  ] [INFO ]  Got 3 search engine results for [SubjectMatterExpert] - 
	 {'https://www.mdpi.com/2077-1444/15/2/213': 'Since the arrival of Christianity in Africa during the pre-colonial era, one of the main characteristics of its spread has been Christian Education (CE). The achievements made thus far by missionaries and African Christian communities were based on the Church-based Christian Education programs that were put into place by churches created by missionaries. Education, let alone Christian ...', 'https://thewitness.org/christianity-in-zimbabwe/': 'The current state of Christianity in Zimbabwe. The current state of Christianity in Zimbabwe is a complex and multifaceted issue. While the country has a strong Christian tradition, with approximately 85% of the population identifying as Christians, there are also numerous challenges facing believers. One major issue is the political climate in ...', 'https://www.britannica.com/topic/Red-Cross-and-

Got search engine results: 5 for [Interactions between Red Cross Church and historical development of Zimbabwe]


2024-03-31 14:42:56,173 [MainThread  ] [INFO ]  Genreted final answer answer="The Red Cross Church, known as the Red Cross in countries under nominally Christian sponsorship, played a significant role in humanitarian efforts globally. Originating from the work of Swiss humanitarian Henri Dunant, the Red Cross has been instrumental in providing aid during crises and disasters. While the Red Cross primarily focuses on humanitarian missions, its work aligns with the broader principles of charity and compassion promoted by Christianity. This aligns with the historical development of Zimbabwe where Christianity, including Christian Education programs established by missionaries, has been influential. The Red Cross Church's emphasis on humanitarian aid resonates with the broader Christian values that have influenced the historical trajectory of Zimbabwe." cited_urls=['https://www.britannica.com/topic/Red-Cross-and-Red-Crescent', 'https://journals.plos.org/globalpublichealth/article?id=10.137

"The Red Cross Church, known as the Red Cross in countries under nominally Christian sponsorship, played a significant role in humanitarian efforts globally. Originating from the work of Swiss humanitarian Henri Dunant, the Red Cross has been instrumental in providing aid during crises and disasters. While the Red Cross primarily focuses on humanitarian missions, its work aligns with the broader principles of charity and compassion promoted by Christianity. This aligns with the historical development of Zimbabwe where Christianity, including Christian Education programs established by missionaries, has been influential. The Red Cross Church's emphasis on humanitarian aid resonates with the broader Christian values that have influenced the historical trajectory of Zimbabwe.\n\nCitations:\n\n[1]: https://www.britannica.com/topic/Red-Cross-and-Red-Crescent\n[2]: https://journals.plos.org/globalpublichealth/article?id=10.1371/journal.pgph.0002799"

In [18]:
example_answer["messages"]

[HumanMessage(content='So you said you were writing an article on {example_topic}?'),
 HumanMessage(content='I apologize for any confusion, but I am actually a Wikipedia writer focused on the early history of Zimbabwe, specifically examining the impact of missionary activities. In this context, I am particularly interested in the interactions between the Red Cross Church and the historical development of Zimbabwe. Can you provide insights into how the Red Cross Church influenced events in Zimbabwe during that time?'),
 AIMessage(content='{"queries": ["Red Cross Church influence on early history of Zimbabwe", "Missionary activities impact on Zimbabwe", "Interactions between Red Cross Church and historical development of Zimbabwe"]}', name='AI'),
 HumanMessage(content='{"https://www.mdpi.com/2077-1444/15/2/213": "Since the arrival of Christianity in Africa during the pre-colonial era, one of the main characteristics of its spread has been Christian Education (CE). The achievements made t

# Construct the Interview Graph

Now that we've defined the editor and domain expert, we can compose them in a graph.


In [19]:
def route_messages(state: InterviewState, name: str = "SubjectMatterExpert"):

    name = cleanup_name(name)

    logger.info(f'Routing messages for [{name}]')

    messages = state["messages"]
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage) and m.name == name]
    )

    if num_responses >= MAX_INTERVIEW_QUESTIONS:
        return END
    
    last_question = messages[-2]
    if last_question.content.endswith("Thank you so much for your help!"):
        return END
    
    logger.info(f'Continue asking question for [{name}] as this is not the last end of the conversation')
    return "ask_question"

In [20]:
builder = StateGraph(InterviewState)

builder.add_node("ask_question", generate_question)
builder.add_node("answer_question", gen_answer)
builder.add_conditional_edges("answer_question", route_messages)
builder.add_edge("ask_question", "answer_question")

builder.set_entry_point("ask_question")
interview_graph = builder.compile().with_config(run_name="Conduct Interviews")

In [21]:
from IPython.display import Image

# comment out if you have not installed pygraphviz
# Image(interview_graph.get_graph().draw_png())

In [22]:

final_step = None

initial_state = {
    "editor": perspectives.editors[0],
    "messages": [
        AIMessage(
            content=f"So you said you were writing an article on {example_topic}?",
            name="SubjectMatterExpert",
        )
    ],
}
async for step in interview_graph.astream(initial_state):
    name = next(iter(step))
    logger.info(f"Processing step: {name}")
    logger.debug("-- ", str(step[name]["messages"])[:300])
    if END in step:
        final_step = step

2024-03-31 14:42:56,264 [MainThread  ] [INFO ]  Generating question for DrTendaiMoyo
2024-03-31 14:42:56,274 [asyncio_3   ] [INFO ]  Swapping roles for DrTendaiMoyo
2024-03-31 14:42:56,275 [asyncio_3   ] [INFO ]  Converted messages for DrTendaiMoyo while swapping roles: 1 messages
2024-03-31 14:42:57,111 [MainThread  ] [INFO ]  Generated question for DrTendaiMoyo
2024-03-31 14:42:57,119 [MainThread  ] [INFO ]  Processing step: ask_question
2024-03-31 14:42:57,204 [MainThread  ] [INFO ]  START - Generate answers for [SubjectMatterExpert]
2024-03-31 14:42:57,204 [MainThread  ] [INFO ]  Swapping roles for SubjectMatterExpert
2024-03-31 14:42:57,205 [MainThread  ] [INFO ]  Converted messages for SubjectMatterExpert while swapping roles: 2 messages
2024-03-31 14:42:57,933 [MainThread  ] [INFO ]  Got 3 search engine queries for [SubjectMatterExpert] -
	 ['Red Cross Church Zimbabwe early history activities', 'Red Cross Church Zimbabwe initiatives impact on local communities', 'Red Cross Churc

Searching DuckDuckGo for [Red Cross Church Zimbabwe early history activities]
Got search engine results: 5 for [Red Cross Church Zimbabwe early history activities]
Searching DuckDuckGo for [Red Cross Church Zimbabwe initiatives impact on local communities]
Got search engine results: 5 for [Red Cross Church Zimbabwe initiatives impact on local communities]
Searching DuckDuckGo for [Red Cross Church Zimbabwe historical development contributions]


2024-03-31 14:43:00,868 [MainThread  ] [INFO ]  Got 3 search engine results for [SubjectMatterExpert] - 
	 {'https://en.wikipedia.org/wiki/International_Red_Cross_and_Red_Crescent_Movement': 'International Red Crossand Red Crescent Movement. The organized International Red Cross and Red Crescent Movement is a humanitarian movement with approximately 16 million volunteers, members, and staff worldwide. It was founded to protect human life and health, to ensure respect for all human beings, and to prevent and alleviate human suffering.', 'https://www.britannica.com/topic/Red-Cross-and-Red-Crescent': 'The Red Cross is the name used in countries under nominally Christian sponsorship, while Red Crescent (adopted on the insistence of the Ottoman Empire in 1906) is the name used in Muslim countries. Henri Dunant. The Red Cross arose out of the work of Henri Dunant, a Swiss humanitarian, who, at the Battle of Solferino, in June 1859, organized ...', 'https://www.chronicle.co.zw/role-of-mission

Got search engine results: 5 for [Red Cross Church Zimbabwe historical development contributions]


2024-03-31 14:43:03,886 [MainThread  ] [INFO ]  Genreted final answer answer="The Red Cross Church, part of the International Red Cross and Red Crescent Movement, has played a significant role in humanitarian efforts globally. In Zimbabwe, the Zimbabwe Red Cross Society (ZRCS), a branch of the movement, has been actively involved in responding to health crises like cholera outbreaks. For instance, in response to a cholera outbreak in Chiredzi, the ZRCS established a cholera treatment center to support the government's efforts. This demonstrates the organization's commitment to addressing public health emergencies and supporting affected communities. Additionally, the ZRCS has launched emergency appeals to raise funds for humanitarian aid, indicating its ongoing efforts to alleviate human suffering in Zimbabwe. The organization's initiatives have had a tangible impact on local communities by providing essential healthcare services during crises." cited_urls=['https://www.ifrc.org/press-

Searching DuckDuckGo for [Red Cross Church Zimbabwe early history social programs]
Got search engine results: 5 for [Red Cross Church Zimbabwe early history social programs]
Searching DuckDuckGo for [Red Cross Church Zimbabwe educational initiatives]
Got search engine results: 5 for [Red Cross Church Zimbabwe educational initiatives]
Searching DuckDuckGo for [Red Cross Church Zimbabwe community development projects]


2024-03-31 14:43:14,707 [MainThread  ] [INFO ]  Got 3 search engine results for [SubjectMatterExpert] - 
	 {'https://www.ifrc.org/press-release/ifrc-intensifies-response-escalating-cholera-outbreak-zimbabwe-emergency-appeal': 'Harare/Nairobi/Geneva, 17 November 2023 — The International Federation of Red Cross and Red Crescent Societies (IFRC) has launched an emergency appeal for 3 million Swiss Francs to support the Zimbabwe Red Cross Society (ZRCS). The health needs exceed available resources, meaning immediate action to is needed to mitigate the impact on affected communities.', 'https://link.springer.com/chapter/10.1007/978-3-031-30541-2_12': 'Abstract. Since independence in 1980, Zimbabwe has implemented more than ten economic development strategies. These policies ranged from the inward-looking, interventionist strategy to the outward-oriented market-driven focus. All these policies endeavored to create employment in the country among many other objectives and to jostle the econom

Got search engine results: 5 for [Red Cross Church Zimbabwe community development projects]


2024-03-31 14:43:19,443 [MainThread  ] [INFO ]  Genreted final answer answer="The Red Cross Church, through the Zimbabwe Red Cross Society (ZRCS), has been actively involved in humanitarian and developmental initiatives, including addressing social and educational needs within local communities. One example of their efforts is reflected in their involvement in the Global Partnership for Education (GPE) initiatives in Zimbabwe. The GPE grants, with a total value of USD 48.8 million for the period of 2023 to 2026, aim to support educational programs in the country. This partnership underscores the Red Cross Church's commitment to enhancing educational opportunities for the local population and contributing to the broader development of Zimbabwe through education-focused initiatives." cited_urls=['https://www.unicef.org/zimbabwe/press-releases/government-zimbabwe-launches-implementation-new-funding-global-partnership-education'] for [SubjectMatterExpert] - 
	 The Red Cross Church, through

In [23]:
final_state = next(iter(final_step.values()))


In [24]:
final_state

{'messages': [AIMessage(content='So you said you were writing an article on Impact of THE Red Cross Church IN Zimbabwe early history?', name='SubjectMatterExpert'),
  AIMessage(content='Could you please elaborate on the specific activities or initiatives undertaken by the Red Cross Church in Zimbabwe during the early history that had a significant impact on the local communities or the broader historical development of the country?', response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 236, 'total_tokens': 276}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}, name='DrTendaiMoyo'),
  AIMessage(content='So you said you were writing an article on Impact of THE Red Cross Church IN Zimbabwe early history?', name='SubjectMatterExpert'),
  HumanMessage(content='Could you please elaborate on the specific activities or initiatives undertaken by the Red Cross Church in Zimbabwe during the early history that

## Refine Outline

At this point in STORM, we've conducted a large amount of research from different perspectives. It's time to refine the original outline based on these investigations. Below, create a chain using the LLM with a long context window to update the original outline.


In [25]:
refine_outline_prompt = get_chat_prompt_from_prompt_templates([prompts.pmt_s_refine_outline, prompts.pmt_h_refine_outline])

# Using turbo preview since the context can get quite long
refine_outline_chain = get_chain_with_outputparser(refine_outline_prompt, fast_llm, outline_parser)\
    .with_config(run_name="Refine Outline")

# refine_outline_prompt.partial(format_instructions=outline_parser.get_format_instructions()) | long_context_llm | outline_parser

In [26]:
refined_outline = refine_outline_chain.invoke(
    {
        "topic": example_topic,
        "old_outline": initial_outline.as_str,
        "conversations": "\n\n".join(
            f"### {m.name}\n\n{m.content}" for m in final_state["messages"]
        ),
    }
)

In [27]:
logger.info(refined_outline.as_str)

2024-03-31 14:43:27,613 [MainThread  ] [INFO ]  # Impact of The Red Cross Church in Zimbabwe Early History

## Introduction to The Red Cross Church in Zimbabwe

The Red Cross Church, part of the International Red Cross and Red Crescent Movement, has had a significant impact on Zimbabwe's early history. Founded with a mission to protect human life and health, the church's presence in Zimbabwe dates back to pivotal moments in the nation's development.

## Founding and Early Mission

The establishment of The Red Cross Church in Zimbabwe was marked by the dedication of its founders to humanitarian principles. The church's initial mission centered around providing aid, fostering respect for all individuals, and alleviating human suffering within the local communities.

## Key Initiatives and Programs

During its early history in Zimbabwe, The Red Cross Church initiated various impactful programs and activities. These included humanitarian efforts aimed at assisting communities during critic

## Generate Article


In [38]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=25
)


reference_docs = [
    Document(page_content=v, metadata={"source": k})
    for k, v in final_state["references"].items()
]

logger.info(f"Number of references: {len(reference_docs)}")

vectorstore = get_inmemory_db(reference_docs, embeddings)

# Get contents of the references
full_docs = await fetch_pages_from_refs(reference_docs[:])

# Summarize
summaries = summarize_full_docs(fast_llm, example_topic, full_docs)

# f1 = list(chain.from_iterable(summaries.values()))
f1 = summaries.values()
full_split_docs = text_splitter.split_documents(f1)

vectorstore.add_documents(full_split_docs)

retriever = vectorstore.as_retriever()


2024-03-31 15:01:09,175 [MainThread  ] [INFO ]  Number of references: 3


Loading web page from url: https://www.ifrc.org/press-release/ifrc-intensifies-response-escalating-cholera-outbreak-zimbabwe-emergency-appeal


Fetching pages: 100%|##########| 1/1 [00:00<00:00, 17.27it/s]


Loading web page from url: https://zimadvocate.com/2024/01/16/zimbabwe-red-cross-springs-into-action-to-combat-cholera-outbreak-in-chiredzi/


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  7.32it/s]


Loading web page from url: https://www.unicef.org/zimbabwe/press-releases/government-zimbabwe-launches-implementation-new-funding-global-partnership-education


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.93it/s]


Summarizing 1/2 : https://zimadvocate.com/2024/01/16/zimbabwe-red-cross-springs-into-action-to-combat-cholera-outbreak-in-chiredzi/
Summarizing 2/2 : https://www.unicef.org/zimbabwe/press-releases/government-zimbabwe-launches-implementation-new-funding-global-partnership-education


In [39]:
full_split_docs

[Document(page_content="La Società della Croce Rossa dello Zimbabwe sta intensificando i suoi sforzi di risposta per combattere un'epidemia di colera a Chiredzi, in Zimbabwe, a seguito di un recente aumento dei casi. Le autorità sanitarie e le organizzazioni umanitarie stanno lavorando insieme per affrontare la situazione. Il Ministero della Salute e dell'Assistenza all'Infanzia ha riferito un totale di 18.100 casi sospetti di colera e 68 morti confermati in laboratorio in tutto il paese. Inoltre, la città di Chiredzi ha riportato 45 nuovi casi sospetti di colera nell'ultimo aggiornamento. La Società della Croce Rossa dello Zimbabwe sta collaborando con il Ministero della Salute e dell'Assistenza all'Infanzia e altri partner per affrontare le sfide poste", metadata={'source': 'https://zimadvocate.com/2024/01/16/zimbabwe-red-cross-springs-into-action-to-combat-cholera-outbreak-in-chiredzi/', 'title': 'Zimbabwe Red Cross Springs into Action to Combat Cholera Outbreak in Chiredzi', 'descr

In [ ]:
d1 = retriever.invoke("What did the red cross do in Zimbabwe?")
print(d1)

#### Generate Sections

Now you can generate the sections using the indexed docs.


In [ ]:


section_writer_prompt = get_chat_prompt_from_prompt_templates([prompts.pmt_s_section_writer, prompts.pmt_h_section_writer])


async def retrieve(inputs: dict):
    docs = await retriever.ainvoke(inputs["topic"] + ": " + inputs["section"])
    formatted = "\n".join(
        [
            f'<Document href="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )
    return {"docs": formatted, **inputs}

wiki_parser = get_pydantic_parser(WikiSection)

section_writer = (
    retrieve
    | section_writer_prompt.partial(format_instructions=wiki_parser.get_format_instructions())
    | long_context_llm
    | wiki_parser
)

In [ ]:
section = await section_writer.ainvoke(
    {
        "outline": refined_outline.as_str,
        "section": refined_outline.sections[1].section_title,
        "topic": example_topic,
    }
)
print(section.as_str)

#### Generate final article

Now we can rewrite the draft to appropriately group all the citations and maintain a consistent voice.


In [ ]:
prompts.pmt_s_writer = generate_system_chat_prompt("""
You are an expert Wikipedia author. Write the complete wiki article on {topic} using the following section drafts:

{draft}

Strictly follow Wikipedia format guidelines.
""")

prompts.pmt_h_writer = generate_human_chat_prompt("""
Write the complete Wiki article using markdown format. Organize citations using footnotes like "[1]","" avoiding duplicates in the footer. Include URLs in the footer.'
""")


In [ ]:


writer_prompt = get_chat_prompt_from_prompt_templates([prompts.pmt_s_writer, prompts.pmt_h_writer])

writer = writer_prompt | long_context_llm | StrOutputParser()

In [ ]:
for tok in writer.stream({"topic": example_topic, "draft": section.as_str}):
    print(tok, end="")

## Final Flow

Now it's time to string everything together. We will have 6 main stages in sequence:
.

1. Generate the initial outline + perspectives
2. Batch converse with each perspective to expand the content for the article
3. Refine the outline based on the conversations
4. Index the reference docs from the conversations
5. Write the individual sections of the article
6. Write the final wiki

The state tracks the outputs of each stage.


In [ ]:
class ResearchState(TypedDict):
    topic: str
    outline: Outline
    editors: List[Editor]
    interview_results: List[InterviewState]
    # The final sections output
    sections: List[WikiSection]
    article: str

In [ ]:
import asyncio


async def initialize_research(state: ResearchState):
    topic = state["topic"]
    coros = (
        generate_outline_direct.ainvoke({"topic": topic}),
        survey_subjects.ainvoke(topic),
    )
    results = await asyncio.gather(*coros)
    return {
        **state,
        "outline": results[0],
        "editors": results[1].editors,
    }


async def conduct_interviews(state: ResearchState):
    topic = state["topic"]
    initial_states = [
        {
            "editor": editor,
            "messages": [
                AIMessage(
                    content=f"So you said you were writing an article on {topic}?",
                    name="SubjectMatterExpert",
                )
            ],
        }
        for editor in state["editors"]
    ]
    # We call in to the sub-graph here to parallelize the interviews
    interview_results = await interview_graph.abatch(initial_states)

    return {
        **state,
        "interview_results": interview_results,
    }


def format_conversation(interview_state):
    messages = interview_state["messages"]
    convo = "\n".join(f"{m.name}: {m.content}" for m in messages)
    return f'Conversation with {interview_state["editor"].name}\n\n' + convo


async def refine_outline(state: ResearchState):
    convos = "\n\n".join(
        [
            format_conversation(interview_state)
            for interview_state in state["interview_results"]
        ]
    )

    updated_outline = await refine_outline_chain.ainvoke(
        {
            "topic": state["topic"],
            "old_outline": state["outline"].as_str,
            "conversations": convos,
        }
    )
    return {**state, "outline": updated_outline}


async def index_references(state: ResearchState):
    all_docs = []
    for interview_state in state["interview_results"]:
        reference_docs = [
            Document(page_content=v, metadata={"source": k})
            for k, v in interview_state["references"].items()
        ]
        all_docs.extend(reference_docs)
    await vectorstore.aadd_documents(all_docs)
    return state


async def write_sections(state: ResearchState):
    outline = state["outline"]
    sections = await section_writer.abatch(
        [
            {
                "outline": refined_outline.as_str,
                "section": section.section_title,
                "topic": state["topic"],
            }
            for section in outline.sections
        ]
    )
    return {
        **state,
        "sections": sections,
    }


async def write_article(state: ResearchState):
    topic = state["topic"]
    sections = state["sections"]
    draft = "\n\n".join([section.as_str for section in sections])
    article = await writer.ainvoke({"topic": topic, "draft": draft})
    return {
        **state,
        "article": article,
    }

#### Create the graph


In [ ]:
builder_of_storm = StateGraph(ResearchState)

nodes = [
    ("init_research", initialize_research),
    ("conduct_interviews", conduct_interviews),
    ("refine_outline", refine_outline),
    ("index_references", index_references),
    ("write_sections", write_sections),
    ("write_article", write_article),
]
for i in range(len(nodes)):
    name, node = nodes[i]
    builder_of_storm.add_node(name, node)
    if i > 0:
        builder_of_storm.add_edge(nodes[i - 1][0], name)

builder_of_storm.set_entry_point(nodes[0][0])
builder_of_storm.set_finish_point(nodes[-1][0])
storm = builder_of_storm.compile()

In [ ]:
async for step in storm.astr eam(
    {
        "topic": "Building better slack bots using LLMs",
    }
):
    name = next(iter(step))
    print(name)
    logger.info("-- ", str(step[name])[:300])
    if END in step:
        results = step

In [ ]:
article = results[END]["article"]

## Render the Wiki

Now we can render the final wiki page!


In [ ]:
from IPython.display import Markdown

# We will down-header the sections to create less confusion in this notebook
Markdown(article.replace("\n#", "\n##"))